### PolyMarket API Calls Through Their Gamma API:

In [13]:
import requests
import json
import time
from datetime import datetime, timedelta

# Base URL for Polymarket Gamma API
base_url = "https://gamma-api.polymarket.com/markets"

current_time = datetime.now()
ninety_days_from_now = current_time + timedelta(days=90)

# Format dates for Polymarket API (using ISO format)
current_time_str = current_time.strftime("%Y-%m-%dT%H:%M:%SZ")
ninety_days_from_now_str = ninety_days_from_now.strftime("%Y-%m-%dT%H:%M:%SZ")

# Print the dates we're using to debug
print(f"Current time: {current_time_str}")
print(f"90 days from now: {ninety_days_from_now_str}")

# Parameters for the markets endpoint
params = {
    "limit": 1000,  # Limit to 10 markets for brevity
    "active": True,  # Only active markets
    "closed": False,  # Not closed (still accepting orders)
    "end_date_min": current_time_str,  # Only markets that haven't closed yet
    "end_date_max": ninety_days_from_now_str,  # Only markets closing within 90 days
}

try:
    # Make the GET request
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        print('Data returned successfully!')
        with open('polymarket_markets_901days.json', 'w') as json_file:
            json.dump(data, json_file, indent=4, sort_keys=True)
    else:
        print(f"Error: {response.status_code} - {response.text}")
        
except Exception as e:
    print(f"Request failed: {e}")

Current time: 2025-04-26T15:18:26Z
90 days from now: 2025-07-25T15:18:26Z
Data returned successfully!


### Polymarket JSON Data Extraction:

In [14]:
import json
import csv
import os

def extract_polymarket_data(input_filename, output_filename):
    try:
        with open(input_filename, 'r') as file:
            data = json.load(file)
    except Exception as e:
        print(f"Error reading JSON file: {e}")
        return
    
    # Check if 'markets' key exists in the data
    if 'markets' in data:
        markets = data['markets']
    else:
        # If the data is a list of markets directly
        markets = data
    
    # Create CSV file
    with open(output_filename, 'w', newline='', encoding='utf-8') as csv_file:
        # Define CSV headers
        fieldnames = ['title', 'description', 'endDate', 'yesPrice', 'noPrice']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the header row
        writer.writeheader()
        
        # Process each market
        for market in markets:
            try:
                # Extract title (question field)
                title = market.get('question', '')
                
                # Extract description
                description = market.get('description', '')
                
                # Extract end date
                end_date = market.get('endDate', '')
                
                # Extract Yes/No prices
                outcome_prices_str = market.get('outcomePrices', '[]')
                outcome_prices = json.loads(outcome_prices_str)
                
                # Default values in case parsing fails
                yes_price = 'N/A'
                no_price = 'N/A'
                
                # Get Yes/No prices if available
                if len(outcome_prices) >= 2:
                    yes_price = outcome_prices[0]
                    no_price = outcome_prices[1]
                
                # Write data to CSV
                writer.writerow({
                    'title': title,
                    'description': description,
                    'endDate': end_date,
                    'yesPrice': yes_price,
                    'noPrice': no_price
                })
                
            except Exception as e:
                print(f"Error processing market: {e}")
                continue
    
    print(f"Data successfully extracted to {output_filename}")

# Usage example
if __name__ == "__main__":
    input_file = "polymarket_markets_901days.json"  # Change this to your input file
    output_file = "polymarket_markets_data.csv"
    
    extract_polymarket_data(input_file, output_file)

Data successfully extracted to polymarket_markets_data.csv


### Kalshi API Calls With JSON Output File


In [7]:
import requests
import json
import time
from datetime import datetime, timedelta


# Base URL for Kalshi production API
base_url = "https://api.elections.kalshi.com/trade-api/v2/markets"

current_time = int(time.time())  # Current time in seconds since epoch
# The Kalshi API specifically expects timestamps in Unix format (seconds since January 1, 1970)
ninety_days_from_now = int((datetime.now() + timedelta(days=90)).timestamp())
print(f'The current time is: {current_time}')
print(f"90 days from now: {ninety_days_from_now}")

params = {
    "limit": 100,
    "status" : "open",
    "min_close_ts": current_time,
    "max_close_ts": ninety_days_from_now
}

try:
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        print('Data returned successfully!')
        with open('kalshi_markets_closed.json', 'w') as json_file:
            json.dump(data, json_file, indent=4, sort_keys=True)
    else:
        print(f"Error: {response.status_code} - {response.text}")

    
except Exception as e:
    print(f"Request failed: {e}")

The current time is: 1745699810
90 days from now: 1753475810
Data returned successfully!


### Kalshi JSON Data Extraction

In [ ]:
import json
import csv
import os

def extract_kalshi_data(input_filename, output_filename):
    try:
        with open(input_filename, 'r') as file:
            data = json.load(file)
    except Exception as e:
        print(f"Error reading JSON file: {e}")
        return
    
    # Check if 'markets' key exists in the data
    if 'markets' in data:
        markets = data['markets']
    else:
        # If the data is a list of markets directly
        markets = data
    
    # Create CSV file
    with open(output_filename, 'w', newline='', encoding='utf-8') as csv_file:
        # Define CSV headers
        fieldnames = ['title', 'description', 'endDate', 'yesPrice', 'noPrice']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the header row
        writer.writeheader()
        
        # Process each market
        for market in markets:
            try:
                # Extract title
                title = market.get('title', '')
                
                # Extract description (using rules_primary for description)
                description = market.get('rules_primary', '')
                
                # Extract end date (using expiration_time)
                end_date = market.get('expiration_time', '')
                
                # Extract Yes/No prices
                yes_price = market.get('yes_ask', 'N/A')
                no_price = market.get('no_ask', 'N/A')
                
                # Convert price from cents to decimal if it's a number
                try:
                    if yes_price != 'N/A':
                        yes_price = float(yes_price) / 100
                    if no_price != 'N/A':
                        no_price = float(no_price) / 100
                except (ValueError, TypeError):
                    # If conversion fails, keep as is
                    pass
                
                # Write data to CSV
                writer.writerow({
                    'title': title,
                    'description': description,
                    'endDate': end_date,
                    'yesPrice': yes_price,
                    'noPrice': no_price
                })
                
            except Exception as e:
                print(f"Error processing market: {e}")
                continue
    
    print(f"Data successfully extracted to {output_filename}")

if __name__ == "__main__":
    input_file = "kalshi_markets_closed.json"  
    output_file = "kalshi_markets_data.csv"
    
    extract_kalshi_data(input_file, output_file)

Data successfully extracted to kalshi_markets_data.csv


In [ ]:
import pandas as pd

def join_prediction_market_csvs(polymarket_file, kalshi_file, output_file):

    poly_df = pd.read_csv(polymarket_file, encoding='utf-8')
    kalshi_df = pd.read_csv(kalshi_file, encoding='utf-8')
    
    poly_df['platform'] = 'Polymarket'
    kalshi_df['platform'] = 'Kalshi'
    
    column_order = ['platform', 'title', 'description', 'endDate', 'yesPrice', 'noPrice']
    poly_df = poly_df[column_order]
    kalshi_df = kalshi_df[column_order]
    
    combined_df = pd.concat([poly_df, kalshi_df], ignore_index=True)
    combined_df.to_csv(output_file, index=False, encoding='utf-8')
    
    print(f"Successfully joined CSVs and saved to {output_file}")
    print(f"Total records: {len(combined_df)}")
    print(f"Polymarket records: {len(poly_df)}")
    print(f"Kalshi records: {len(kalshi_df)}")


if __name__ == "__main__":
    polymarket_file = "polymarket_markets_data.csv"
    kalshi_file = "kalshi_markets_data.csv"
    combined_file = "combined_prediction_markets.csv"
    
    join_prediction_market_csvs(polymarket_file, kalshi_file, combined_file)

Successfully joined CSVs and saved to combined_prediction_markets.csv
Total records: 200
Polymarket records: 100
Kalshi records: 100


### Prototyping Arbitrage Opportunities



1. **Resolution Date Compatibility**
   - Markets should resolve on the same date or within an acceptable timeframe
   - Different resolution dates introduce timing risk to your arbitrage

2. **Resolution Criteria Alignment**
   - Markets should have matching resolution conditions beyond just similar descriptions
   - Small differences in resolution criteria can lead to different outcomes

3. **Liquidity Assessment**
   - Both markets should have sufficient liquidity to execute trades
   - Low liquidity could prevent closing positions or cause price slippage

4. **Fee Impact Analysis**
   - Transaction fees on both platforms should be factored into profit calculations
   - Withdrawal and deposit fees can significantly reduce arbitrage profitability

5. **Execution Speed Considerations**
   - Time between identifying opportunity and executing trades matters
   - Prices can shift during execution, especially in volatile markets

6. **Platform-Specific Rules**
   - Market closure policies should align (some markets can close early)
   - Different handling of "edge cases" in each platform's rules

7. **Capital Lockup Duration**
   - Consider how long capital will be tied up in the position
   - Shorter resolution timeframes may be preferable for capital efficiency

8. **Market Correlation Verification**
   - Confirm the markets are truly about the same event
   - Watch for subtle differences in wording that change the actual event being predicted

For prototyping I will just focus on:

1. Contract Name
2. Contract Resolution Description
3. Spread Of Contracts